# Upload nuclei/soma table to CAVE
## Prepare CAVE environment

In [1]:
import numpy as np
import sys
import os
import pandas as pd
from datetime import datetime
from caveclient import CAVEclient

sys.path.append(os.path.abspath("../segmentation"))# to import rootID_lookup and authentication_utils like below
import rootID_lookup as IDlook
import authentication_utils as auth

In [2]:
# auth = client.auth
# auth.get_new_token()
# new_token = '' # This is the text you see after you visit the website.
# auth.save_token(token=new_token)

In [3]:
# check datastack_name here https://globalv1.daf-apis.com/info/
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.info.get_datastack_info()

{'description': 'This is the first version of the fanc dataset hosted in CAVE.',
 'aligned_volume': {'image_source': 'precomputed://gs://zetta_lee_fly_vnc_001_precomputed/fanc_v4_em',
  'name': 'fanc_v4',
  'description': 'This is the 4th alignment of the female adult nerve cord dataset.',
  'id': 7},
 'synapse_table': None,
 'segmentation_source': 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod',
 'analysis_database': None,
 'local_server': 'https://cave.fanc-fly.com',
 'soma_table': None,
 'viewer_site': None}

In [4]:
client.annotation.get_tables()

https://cave.fanc-fly.com/annotation/api/v2/aligned_volume/fanc_v4/table


['t1mn', 'test_soma', 'synapses_may2021', 'nuclei_aug2021', 'soma_aug2021']

## format tables

In [5]:
df = pd.read_csv('../Output/soma_info_Aug2021ver2.csv', header=0)

In [6]:
# https://globalv1.daf-apis.com/schema/views/type/nucleus_detection/view
nuclei_df = df.reindex(columns=['nucID', 'nuc_xyz', 'bbx_min', 'bbx_max'])
nuclei_df = nuclei_df.rename(columns={'nucID': 'id', 'nuc_xyz': 'pt_position', 'bbx_min': 'bb_start_position', 'bbx_max': 'bb_end_position'})
nuclei_df['type'] = 'nucleus_detection'

soma_df = df.reindex(columns=['nucID', 'soma_xyz'])
soma_df = soma_df.rename(columns={'nucID': 'id', 'soma_xyz': 'pt_position'})
soma_df['type'] = 'nucleus_detection'

In [7]:
nuclei_df_pt = nuclei_df['pt_position'].str.strip('()').str.split(',',expand=True)
nuclei_df['pt_position'] = nuclei_df_pt.astype(int).values.tolist()

nuclei_df_bb_start = nuclei_df['bb_start_position'].str.strip('()').str.split(',',expand=True)
nuclei_df['bb_start_position'] = nuclei_df_bb_start.astype(int).values.tolist()

nuclei_df_bb_end = nuclei_df['bb_end_position'].str.strip('()').str.split(',',expand=True)
nuclei_df['bb_end_position'] = nuclei_df_bb_end.astype(int).values.tolist()

In [8]:
soma_df_pt = soma_df['pt_position'].str.strip('()').str.split(',',expand=True)
soma_df['pt_position'] = soma_df_pt.astype(int).values.tolist()

In [17]:
# check tables
nuclei_df

,id,pt_position,bb_start_position,bb_end_position,type
0,73115668492321345,"[64384, 155376, 2719]","[64144, 155296, 2711]","[64544, 155760, 2756]",nucleus_detection
1,72341681025843555,"[16960, 157296, 2652]","[16832, 157232, 2639]","[17216, 157648, 2692]",nucleus_detection
2,72694142819371754,"[40352, 195888, 1211]","[40224, 195760, 1197]","[40672, 196240, 1238]",nucleus_detection
3,72904562259788369,"[52752, 154288, 2630]","[52368, 153920, 2624]","[52848, 154320, 2671]",nucleus_detection
4,72481250081768943,"[27376, 86896, 2005]","[27008, 86544, 1982]","[27408, 87104, 2023]",nucleus_detection
...,...,...,...,...,...
17071,73114912242532398,"[60624, 109584, 1417]","[59856, 108960, 1351]","[62560, 112032, 1522]",nucleus_detection
17072,72903943314735186,"[48496, 116896, 969]","[47664, 116336, 828]","[50896, 118912, 1003]",nucleus_detection
17073,72270762727178308,"[14112, 123456, 3312]","[13216, 123328, 3093]","[15488, 125712, 3363]",nucleus_detection
17074,72200462366935570,"[8176, 127328, 2290]","[8080, 126608, 2245]","[10064, 129536, 2501]",nucleus_detection


## Update annotation table

In [ ]:
upload/delete

## Upload annotation table
make sure to create tables first

https://github.com/seung-lab/CAVEclient/blob/72ebf7180ca3c4195bec561e177c453554cc60c6/FlyWireAnnotationTutorial.ipynb

In [1]:
client.schema.schema_definition('nucleus_detection')

NameError: name 'client' is not defined

In [18]:
# get 502 error if you try to upload at the same time
# nuclei_aug2021
k=100
minidfs = [nuclei_df.loc[i:i+k-1, :] for i in range(0, len(nuclei_df), k)]
for dftmp in minidfs:
#    client.annotation.post_annotation_df(table_name='nuclei_aug2021',
                                         df=dftmp,
                                         position_columns={'pt_position': 'pt', 'bb_start_position': 'bb_start', 'bb_end_position': 'bb_end'}, 
                                         aligned_volume_name='fanc_v4')

In [16]:
# soma_aug2021
k=100
minidfs = [soma_df.loc[i:i+k-1, :] for i in range(0, len(soma_df), k)]
for dftmp in minidfs:
#    client.annotation.post_annotation_df(table_name='soma_aug2021',
                                         df=dftmp,
                                         position_columns=['pt_position'], 
                                         aligned_volume_name='fanc_v4')

## Create tables

https://github.com/seung-lab/CAVEclient/blob/dec850dff9e0fa0a9ef8c2a0c79bf82cd2fc5b97/caveclient/annotationengine.py

In [29]:
print('IDs shared with soma_aug2021. Based on nuclei segmented at {}'.format(auth.get_cv_path('nuclei_seg_Aug2021')['url']))

IDs shared with soma_aug2021. Based on nuclei segmented at precomputed://gs://ng_scratch_ranl_zetta_30/vnc1_full_v3align_2/realigned_v1/seg/84214f67075692a9e41c02d164b466ba


In [30]:
# client.annotation.create_table(table_name='nuclei_aug2021',
                               schema_name='nucleus_detection',
                               description = 'IDs shared with soma_aug2021. Based on nuclei segmentation at {}'.format(auth.get_cv_path('nuclei_seg_Aug2021')['url']),
                               voxel_resolution =[4.3, 4.3, 45.0])

<Response [200]>

In [31]:
# client.annotation.create_table(table_name='soma_aug2021',
                               schema_name='nucleus_detection',
                               description = 'IDs shared with nuclei_aug2021. Based on nuclei segmentation at {}'.format(auth.get_cv_path('nuclei_seg_Aug2021')['url']),
                               voxel_resolution =[4.3, 4.3, 45.0])

<Response [200]>

## Test materialization

In [5]:
client.materialize.homepage

In [16]:
client.materialize.get_versions()

[18, 19, 20, 21]

In [19]:
# client.materialize.version = 21

In [6]:
client.materialize.get_annotation_count('soma_aug2021')

KeyError: 'version'

In [19]:
test=client.materialize.live_query('soma_aug2021',
                                   datetime.utcnow(),
                                   limit=20)

In [20]:
test

,id,valid,pt_supervoxel_id,pt_root_id,volume,pt_position,bb_start_position,bb_end_position
0,73115668492321345,t,74244180477821787,648518346472873828,NaN,"[64096, 155520, 2735]","[nan, nan, nan]","[nan, nan, nan]"
1,72341681025843555,t,72625768014247001,648518346496058999,NaN,"[16768, 157376, 2693]","[nan, nan, nan]","[nan, nan, nan]"
2,72694142819371754,t,73401129131935868,648518346511627376,NaN,"[40144, 195856, 1213]","[nan, nan, nan]","[nan, nan, nan]"
3,72904562259788369,t,73821967945813511,648518346504953141,NaN,"[52320, 153968, 2663]","[nan, nan, nan]","[nan, nan, nan]"
4,72481250081768943,t,72975274937264610,648518346499742019,NaN,"[26928, 86432, 1996]","[nan, nan, nan]","[nan, nan, nan]"
5,73045230693122208,t,74032867348432337,648518346512714794,NaN,"[57920, 148672, 1348]","[nan, nan, nan]","[nan, nan, nan]"
6,73115599638626701,t,74173605239618700,648518346475960175,NaN,"[62560, 149344, 2111]","[nan, nan, nan]","[nan, nan, nan]"
7,72341886915837985,t,72696548471049575,648518346512656677,NaN,"[19376, 169888, 1543]","[nan, nan, nan]","[nan, nan, nan]"
8,72834605496926428,t,73682054486949216,648518346518927109,NaN,"[48608, 179712, 1440]","[nan, nan, nan]","[nan, nan, nan]"
9,72411362843689699,t,72765131649878154,648518346508294966,NaN,"[20624, 115744, 3661]","[nan, nan, nan]","[nan, nan, nan]"


In [17]:
test=client.materialize.live_query('soma_aug2021',
                                   datetime.utcnow(),
                                   filter_in_dict={'id': [73115668492321345]},
                                   limit=10)

In [18]:
test

,id,valid,pt_supervoxel_id,pt_root_id,volume,pt_position,bb_start_position,bb_end_position
0,73115668492321345,t,74244180477821787,648518346472873828,NaN,"[64096, 155520, 2735]","[nan, nan, nan]","[nan, nan, nan]"
